# Train CNN Models

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from model.dataset import SpectraDataset, DataGenerator

2022-12-28 13:52:20.599138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


## Prepare Dataset

In [2]:
data_dir = 'data'
dataset = SpectraDataset(data_dir)

Reading train data
No. of materials : 22
[['Acrylic' '6']
 ['ENR' '6']
 ['EPDM' '6']
 ['HDPE' '6']
 ['LDPE' '6']
 ['Nylon' '6']
 ['PBAT' '6']
 ['PBS' '6']
 ['PC' '6']
 ['PEEK' '6']
 ['PEI' '6']
 ['PET' '6']
 ['PLA' '6']
 ['PMMA' '6']
 ['POM' '6']
 ['PP' '6']
 ['PS' '6']
 ['PTEE' '6']
 ['PU' '6']
 ['PVA' '6']
 ['PVC' '6']
 ['cellulose' '6']]
Read 132 spectra for training.
No. of data points/shifts per spectra (input shape) : (1802,)
No. of classes : 22


In [3]:
dataset.get_binary_targets().shape

(132, 22)

In [4]:
dataset.get_training_data().shape

(132, 1802)

In [5]:
datagen = DataGenerator(dataset)

## Train

In [6]:
input_shape = dataset.get_input_shape()
num_class = dataset.get_num_class()

### ANN

In [7]:
# from model.nn import ANN

# ann = ANN(input_shape, num_class)
# ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# ann.summary()


# ann.fit(datagen, epochs=200)
# # ann.fit(x=dataset.get_training_data(), 
# #         y=dataset.get_binary_targets(), 
# #         epochs=50, 
# #         batch_size=32) #optimal

### AlexNet

In [8]:
# from model.nn import AlexNet

# alexnet = AlexNet(input_shape, num_class)
# alexnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# alexnet.summary()

# alexnet.fit(datagen, epochs=200)

### VGG16

In [9]:
# from model.nn import VGG16

# vgg16 = VGG16(input_shape, num_class)
# vgg16.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# vgg16.summary()

# vgg16.fit(datagen, epochs=200, validation_data=dataset.get_test_data())

### ResNet50

In [10]:
# from model.nn import ResNet50

# resnet = ResNet50(input_shape, num_class)
# resnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# resnet.summary()

# resnet.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

### DenseNet

In [18]:
from model.nn import DenseNet

densenet = DenseNet(input_shape, num_class)
densenet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
densenet.summary()

densenet.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 1802, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 1802, 36)     108         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 1802, 36)     144         conv1d_6[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 1802, 36)     0           batch_normalization_2[0][0]      
______________________________________________________________________________________________

2022-12-28 13:56:56.030587: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-28 13:56:56.049926: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3692675000 Hz
2022-12-28 13:57:05.294232: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-12-28 13:57:05.455266: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


4/4 [==============================] - 14s 627ms/step - loss: 0.5782 - binary_accuracy: 0.9351 - val_loss: 0.5315 - val_binary_accuracy: 0.9545
Epoch 2/20
4/4 [==============================] - 1s 177ms/step - loss: 0.3710 - binary_accuracy: 0.9473 - val_loss: 0.3875 - val_binary_accuracy: 0.9566
Epoch 3/20
4/4 [==============================] - 1s 177ms/step - loss: 0.2313 - binary_accuracy: 0.9553 - val_loss: 0.3687 - val_binary_accuracy: 0.9582
Epoch 4/20
4/4 [==============================] - 1s 178ms/step - loss: 0.2016 - binary_accuracy: 0.9620 - val_loss: 0.4610 - val_binary_accuracy: 0.9545
Epoch 5/20
4/4 [==============================] - 1s 181ms/step - loss: 0.1645 - binary_accuracy: 0.9644 - val_loss: 0.5043 - val_binary_accuracy: 0.9545
Epoch 6/20
4/4 [==============================] - 1s 179ms/step - loss: 0.1443 - binary_accuracy: 0.9642 - val_loss: 0.4857 - val_binary_accuracy: 0.9545
Epoch 7/20
4/4 [==============================] - 1s 179ms/step - loss: 0.1207 - binar

### LSTM

In [ ]:
from model.rnn import VanLSTM

lstm = VanLSTM(input_shape, num_class)
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
lstm.summary()

lstm.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

## Test

In [19]:
from utils import multilabel_classification_report

In [20]:
# multilabel_classification_report(dataset, ann)

In [21]:
# multilabel_classification_report(dataset, alexnet)

In [22]:
# multilabel_classification_report(dataset, vgg16)

In [23]:
# multilabel_classification_report(dataset, resnet)

In [24]:
multilabel_classification_report(dataset, densenet)

Reading test data
Read 88 spectra for testing.
3/3 [==============================] - 0s 32ms/step - loss: 0.2915 - binary_accuracy: 0.9545
Loss: 0.2915, Binary Accuracy: 0.9545
            precision     recall   f1-score    support
Acrylic          0.00       0.00       0.00          4
ENR              0.00       0.00       0.00          4
EPDM             0.00       0.00       0.00          4
HDPE             0.00       0.00       0.00          4
LDPE             0.00       0.00       0.00          4
Nylon            0.00       0.00       0.00          4
PBAT             0.00       0.00       0.00          4
PBS              0.00       0.00       0.00          4
PC               0.00       0.00       0.00          4
PEEK             0.00       0.00       0.00          4
PEI              0.00       0.00       0.00          4
PET              0.00       0.00       0.00          4
PLA              0.00       0.00       0.00          4
PMMA             0.00       0.00       0.00         

/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/an